# Customer Analysis

In [4]:
# Importing required libraries for data loading, inspection and initial cleaning.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns


In [5]:
# Importing dataset

raw_path = "data/multi-dimensional-contact-segme.csv"

df = pd.read_csv(raw_path)
df.head()

,Full Name,Create Date,Email,Industry,Website URL,Country/Region,Annual Revenue,Lifecycle Stage,Original Source,Original Source Drill-Down 1,Deal Stage,Closed amount
0,Darren Lindsay,21/10/2022 11:06,darren@agmchilleddistribution.co.uk,Logistics,agmchilleddistribution.co.uk,United Kingdom,(No value),Opportunity,Paid Search,cold stores uk,(No value),(No value)
1,Grady Broadhurst,18/10/2022 10:58,gradyb@hotmail.co.uk,(No value),(No value),United Kingdom,(No value),Lead,Paid Search,blast freezer uk,(No value),(No value)
2,Graham hall,17/10/2022 14:09,info@hallsofhazlemere.com,(No value),hallsofhazlemere.com,England,(No value),Lead,Paid Search,refrigerated containers,(No value),(No value)
3,Kate Sampson,25/10/2022 15:09,kate@copas.co.uk,Meat,copas.co.uk,(No value),10000000,Lead,Direct Traffic,www.crscoldstorage.co.uk/refrigeration/refrige...,(No value),(No value)
4,martin rowland,26/10/2022 09:18,martinrowland30@gmail.com,Hospitality,(No value),United Kingdom,(No value),Customer,Paid Search,cold stores uk,Closed Won (Salesforce - Opportunity),9320
